<a href="https://colab.research.google.com/github/monicman/dataengineering/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 49kB/s 
     |████████████████████████████████| 204kB 38.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=c7f5d695ad6b565951c7099651e909abc73a28d0e7244029c26f5a239a8301e5
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import urllib
from urllib.request import urlopen   
import io
import requests
from zipfile import ZipFile
import os

In [0]:
spark = SparkSession.builder.getOrCreate()

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    print("error here")
    df = pd.read_csv(filename,sep='\t')
    print("no actually here")
    return spark.createDataFrame(df)

In [0]:
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    df = pd.read_csv(filename,sep='\t')
    df.iloc[:, 3:6] = df.iloc[:, 3:6].apply(pd.to_datetime, errors='coerce', format='%d/%m/%Y')
    myschema = StructType([StructField('REGISTER_NAME', StringType(), True),
                         StructField('BN_NAME', StringType(), True),
                         StructField('BN_STATUS', StringType(), True),
                         StructField('BN_REG_DT', DateType(), True),
                         StructField('BN_CANCEL_DT', DateType(), True),
                         StructField('BN_RENEW_DT', DateType(), True),
                         StructField('BN_STATE_NUM', StringType(), True),
                         StructField('BN_STATE_OF_REG', StringType(), True),
                         StructField('BN_ABN', DoubleType(), True)])
    return spark.createDataFrame(df, schema=myschema)

In [0]:
#ipgod101 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/e5cbeafc-5fb3-4dfd-bd22-afe81b6ab1e1/download/ipgod101.csv'
#ipgod102 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/846990df-db42-4ad7-bbd6-567fd37a2797/download/ipgod102.csv'  
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'

#staging_ipgod101 = ingest_files(ipgod101,'ipgod101.csv')
#staging_ipgod102 = ingest_files(ipgod102,'ipgod102.csv')
staging_abr = ingest_files(ABN,'zipFile.zip')

In [12]:
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'
staging_abr.head(10)

[Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='            Melbourne Collision Repair Centre Mentone', BN_STATUS='Registered', BN_REG_DT=datetime.date(2019, 5, 3), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2020, 5, 3), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=84263023451.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='  Bruce Ward Training', BN_STATUS='Registered', BN_REG_DT=datetime.date(2018, 4, 12), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2020, 4, 12), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=16897173642.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='  Class Act Management', BN_STATUS='Registered', BN_REG_DT=datetime.date(2017, 10, 4), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2020, 10, 4), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=31892662187.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='  i Prep', BN_STATUS='Deregistered', BN_REG_DT=datetime.date(2015, 8, 5), BN_CANCEL_DT=datetime.date(2018, 12, 13), BN_RENEW_DT=datetime.date(2018, 8, 5

In [15]:
staging_ipgod101.head(5)

[Row(australian_appl_no='2018904968', application_year='2018', sealing_year=None, applicant_count='1', australian='True', foreign='False', entity='False', non_entity='True', patent_status_type='FILED', patent_type='Provisional', primary_ipc_mark_value=None),
 Row(australian_appl_no='2017215797', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_type='Standard', primary_ipc_mark_value='E06B   9/00'),
 Row(australian_appl_no='2017216336', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_type='Standard', primary_ipc_mark_value='A61M  31/00'),
 Row(australian_appl_no='2017216324', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_ty

In [16]:
staging_ipgod102.head(5)

[Row(australian_appl_no='1904000001', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000004', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000006', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000009', ipa_i

In [0]:
staging_ipgod101.createOrReplaceTempView('staging_ipgod102')
staging_ipgod102.createOrReplaceTempView('staging_ipgod102')

region_table = spark.sql("""SELECT DISTINCT
                                       lat,
                                       lon,
                                       sa2_code,
                                       sa2_name,
                                       lga_code,
                                       lga_name,
                                       gcc_name,
                                       elect_div,
                                       elect_div
                                       userId
                                   FROM staging_ipgod102
                                   where australian = 'True' and entity = 'True'
                                   order by lat,lon
                                   """) 

In [26]:
region_table.head(5)

[Row(lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, userId=None),
 Row(lat='-12.051', lon='134.233', sa2_code='702041063.0', sa2_name='East Arnhem', lga_code='71300.0', lga_name='East Arnhem (R)', gcc_name='Rest of NT', elect_div='Lingiari', userId='Lingiari'),
 Row(lat='-12.182', lon='136.782', sa2_code='702041064.0', sa2_name='Nhulunbuy', lga_code='79399.0', lga_name='Unincorporated NT', gcc_name='Rest of NT', elect_div='Lingiari', userId='Lingiari'),
 Row(lat='-12.333', lon='136.817', sa2_code='702041063.0', sa2_name='East Arnhem', lga_code='71300.0', lga_name='East Arnhem (R)', gcc_name='Rest of NT', elect_div='Lingiari', userId='Lingiari'),
 Row(lat='-12.355', lon='130.880', sa2_code='701021027.0', sa2_name='Tiwi', lga_code='71000.0', lga_name='Darwin (C)', gcc_name='Greater Darwin', elect_div='Solomon', userId='Solomon')]

In [0]:
region_table =  region.select("start_time").distinct() \
        .withColumn("hour", hour("start_time")).withColumn("day", dayofmonth("start_time")) \
        .withColumn("week", weekofyear("start_time")).withColumn("month", month("start_time")) \
        .withColumn("year", year("start_time")).withColumn("weekday", dayofweek("start_time"))
